In [80]:
# For data processing
import pandas as pd
import re
from collections import defaultdict
# For file management/retrieval/download
from urllib.request import urlretrieve
import wget
import time
import pickle
import glob
import  tarfile
# pretty print
from pprint import pprint
# Error
import traceback

In [38]:
with open("../../../PaperAcceptancePrediction/shruti/features/iclr_arxiv_map.pkl", "rb") as f:
    iclr_arxiv_map = pickle.load(f)
len(iclr_arxiv_map)

4897

In [117]:
files = glob.glob("/home/singh_shruti/workspace/ICLR_arxiv_dump/*")
print("Total files: ", len(files))

Total files:  2506


In [57]:
arxiv_source_files = [f.rsplit("/", 1)[1] for f in files]
len(arxiv_source_files)

2498

In [42]:
# Find papers with arxiv_id but no source files
for k, v in iclr_arxiv_map.items():
    if v["found"] and not v["arxivId"] in arxiv_source_files:
        #print(k)
        print(v["arxivId"])

1802.04948v1
1711.01970v1
1806.04640v1
1812.11240v2
1805.09980v1
1810.06544v1
1805.09208v1
1909.05352v2


In [20]:
c = 0
for k, v in iclr_arxiv_map.items():
    if v["found"]:
        c += 1
c

2506

In [21]:
c = 0
# Find papers with arxiv_id but no source files
for k, v in iclr_arxiv_map.items():
    if v["found"] and v["arxivId"] in arxiv_source_files:
        c+= 1
print(c)

2504

In [24]:
# find errs of incorrect matchings?
rev_map = defaultdict(list)
for k, v in iclr_arxiv_map.items():
    if v["found"]:
        rev_map[v["arxivId"]].append(k)

print("Ideally acc to 1-1 matching, count should be 2504, but is:", len(rev_map))
        
for k in rev_map:
    if len(rev_map[k]) > 1:
        print(k, rev_map[k])

Ideally acc to 1-1 matching, count should be 2504, but is: 2494
1802.04412v4 ['2018_Bk6qQGWRb', '2019_B1e7hs05Km']
1806.03852v4 ['2018_ByJWeR1AW', '2020_H1eqOnNYDH']
1706.01566v4 ['2018_HyBbjW-RW', '2019_SJf_XhCqKm']
1810.05934v4 ['2018_S1Y7OOlRZ', '2019_S1MAriC5F7']
1802.04948v3 ['2018_SJvu-GW0b', '2019_Ske7ToC5Km']
1711.01970v2 ['2018_SyBBgXWAZ', '2019_BklCusRct7']
1812.10607v1 ['2019_Bkeuz20cYm', '2020_ByedzkrKvH']
1806.04640v2 ['2019_H1eRBoC9FX', '2020_S1et1lrtwr']
1907.03179v1 ['2019_S14h9sCqYm', '2020_SygfNCEYDH']
1805.09980v2 ['2019_SJz6MnC5YQ', '2020_r1e0G04Kvr']
1810.06544v4 ['2019_SyehMhC9Y7', '2020_Skl4mRNYDr']
1805.09208v2 ['2019_rklwwo05Ym', '2020_rJxwDTVFDB']


In [37]:
#inspecting the double matched papers:
# 1812.10607v1 ['2019_Bkeuz20cYm', '2020_ByedzkrKvH']
print(iclr_arxiv_map["2019_S14h9sCqYm"])
print("\n")
print(iclr_arxiv_map["2020_SygfNCEYDH"])

{'id': '2019_S14h9sCqYm', 'title': 'Weakly-supervised Knowledge Graph Alignment with Adversarial Learning', 'abstract': 'Aligning knowledge graphs from different sources or languages, which aims to align both the entity and relation, is critical to a variety of applications such as knowledge graph construction and question answering. Existing methods of knowledge graph alignment usually rely on a large number of aligned knowledge triplets to train effective models. However, these aligned triplets may not be available or are expensive to obtain for many domains. Therefore, in this paper we study how to design fully-unsupervised methods or weakly-supervised methods, i.e., to align knowledge graphs without or with only a few aligned triplets. We propose an unsupervised framework based on adversarial training, which is able to map the entities and relations in a source knowledge graph to those in a target knowledge graph. This framework can be further seamlessly integrated with existing su

# Now all duplicate papers resolved and downloaded all versions

## Plot counts

In [43]:
yearwise_found = {k:defaultdict(int) for k in [2017, 2018, 2019, 2020]} 

for k in iclr_arxiv_map:
    if iclr_arxiv_map[k]["found"]:
        year = int(k.split("_")[0])
        yearwise_found[year][iclr_arxiv_map[k]["label"]] += 1

In [45]:
pprint(yearwise_found)

{2017: defaultdict(<class 'int'>, {'Reject': 128, 'Accept': 170}),
 2018: defaultdict(<class 'int'>, {'Accept': 287, 'Reject': 192}),
 2019: defaultdict(<class 'int'>, {'Reject': 348, 'Accept': 406}),
 2020: defaultdict(<class 'int'>, {'Accept': 385, 'Reject': 588})}


## Pretty print

| Year | Found:Rejected | Found:Accepted |
| --- | --- | --- |
| 2017 | 128/245 | 170/195 |
| 2018 | 192/486 | 287/336 |
| 2019 | 348/917 | 406/502 |
| 2020 | 588/1526 | 385/687 |
| **Total** | 1256/3174 | 1248/1720 |

In [118]:
count = 0
not_count = 0
err = 0

not_count_list = []
err_list = []

for f in files:
    try:
        if not f.endswith("tmp"):
            tar = tarfile.open(f)
            file_mems = tar.getmembers()
            found = False

            for t in file_mems:
                if t.name.find(".bib") > -1:
                    found = True
                    #bib += 1
                    break
                if t.name.find(".bbl") > -1:
                    found = True
                    #bbl += 1
                    break
            if found:
                count +=1
            else:
                not_count +=1
                not_count_list.append(f)
    except Exception as ex:
        err += 1
        err_list.append(f)

In [119]:
bib, bbl, count, not_count, err

(0, 0, 2362, 131, 13)

In [120]:
print(err_list[0:5])
print(not_count_list[0:5])

['/home/singh_shruti/workspace/ICLR_arxiv_dump/1810.00953v4', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1906.05827v3', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1903.03107v2', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1804.10200v1', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1910.02109v1']
['/home/singh_shruti/workspace/ICLR_arxiv_dump/1810.01588v1', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1610.06918v1', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1909.13111v1', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1804.02391v2', '/home/singh_shruti/workspace/ICLR_arxiv_dump/1611.02247v3']


In [52]:
# Counting bib files
mul = 0
err = 0

for f in files:
    try:
        if not f.endswith("tmp"):
            tar = tarfile.open(f)
            file_mems = tar.getmembers()
            found = False
            
            count = 0
            for t in file_mems:
                if t.name.find(".bib") > -1:
                    mul += 1
                    break
    except Exception as ex:
        err += 1

In [53]:
mul, err

(659, 13)

In [54]:
mul = 0
err = 0

for f in files:
    try:
        if not f.endswith("tmp"):
            tar = tarfile.open(f)
            file_mems = tar.getmembers()
            found = False
            
            count = 0
            for t in file_mems:
                if t.name.find(".bbl") > -1:
                    mul += 1
                    break
    except Exception as ex:
        err += 1

In [55]:
mul, err

(2353, 13)

# Saving the bib information

In [56]:
import bibtexparser

In [58]:
files = glob.glob("/home/singh_shruti/workspace/ICLR_arxiv_dump/*")
arxiv_source_files = [f.rsplit("/", 1)[1] for f in files]
len(arxiv_source_files)

2506

In [113]:
bib1_dict = {}

for k, v in iclr_arxiv_map.items():
    if v["found"] and v["arxivId"] in arxiv_source_files:
        try:
            f = "/home/singh_shruti/workspace/ICLR_arxiv_dump/" + v["arxivId"]
            tar = tarfile.open(f)
            file_mems = tar.getmembers()
            
            bib_files = []
            
            for t in file_mems:
                if t.name.find(".bib") > -1:
                    bib_files.append(t.name)
            
            if bib_files:
                if len(bib_files) == 1:
                    
                    tar.extract(member=bib_files[0], path="/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison/temp_ext_dir/")
                    
                    new_loc = "/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison/temp_ext_dir/" + bib_files[0]
                    with open(new_loc) as bibtex_file:
                        bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)
                        #bibtex_str = bibtex_file.read()
                    #bib_database = bibtexparser.loads(bibtex_str)
                    
                    bib_entries = []
                    for i, e in enumerate(bib_database.entries):
                        e["seq"] = i
                        bib_entries.append(e)
                    bib1_dict[k] = bib_entries
#                 else:
                    #read multiple
        except Exception as ex:
            print(k, ex)
#             print(ex)
#             if ex != "file could not be opened successfully":
#                 print(repr(traceback.print_stack()))


2017_HyAddcLge file could not be opened successfully
2017_r1osyr_xg file could not be opened successfully


Entry type unknown not standard. Not considered.
Entry type online not standard. Not considered.


2018_rkeZRGbRW file could not be opened successfully
2019_B1e0X3C9tQ 'f'
2019_B1x9siCcYQ 'june'
2019_BkedwoC5t7 file could not be opened successfully
2019_BkgFqiAqFX 'february'


Entry type online not standard. Not considered.
Entry type online not standard. Not considered.


2019_HJMXTsCqYQ 'jacsat'


...

Entry type software not standard. Not considered.


2020_rJeBJJBYDB 'december'


Overwritting existing string for key: amstrans.
Overwritting existing string for key: bullams.
Overwritting existing string for key: procams.
Overwritting existing string for key: transams.
Overwritting existing string for key: cacm.
Overwritting existing string for key: acmmathsoft.
Overwritting existing string for key: signum.
Overwritting existing string for key: amerstatassoc.
Overwritting existing string for key: applmathcomp.
Overwritting existing string for key: amermathmonthly.
Overwritting existing string for key: britstatpsych.
Overwritting existing string for key: canmathbull.
Overwritting existing string for key: compapplmath.
Overwritting existing string for key: compphys.
Overwritting existing string for key: compstruct.
Overwritting existing string for key: compjour.
Overwritting existing string for key: compsyssci.
Overwritting existing string for key: contempmath.
Overwritting existing string for key: giornalemath.
Overwritting existing string for key: ieeetranscomp.
O

2020_rkeeoeHYvr 'scholarpedia'


Entry type online not standard. Not considered.
Entry type patent not standard. Not considered.
Entry type online not standard. Not considered.


In [114]:
with open("bib_dictionary.pkl", "wb") as f:
    pickle.dump(bib_dict, f)

In [116]:
bib_dict["2020_B1eXygBFPH"]

[{'year': '2016',
  'journal': 'arXiv preprint arXiv:1607.02533',
  'author': 'Kurakin, Alexey and Goodfellow, Ian and Bengio, Samy',
  'title': 'Adversarial examples in the physical world',
  'ENTRYTYPE': 'article',
  'ID': 'kurakin2016adversarial',
  'seq': 0},
 {'organization': 'IEEE',
  'year': '2017',
  'pages': '39--57',
  'booktitle': '2017 IEEE Symposium on Security and Privacy (SP)',
  'author': 'Carlini, Nicholas and Wagner, David',
  'title': 'Towards evaluating the robustness of neural networks',
  'ENTRYTYPE': 'inproceedings',
  'ID': 'carlini2017towards',
  'seq': 1},
 {'organization': 'ACM',
  'year': '2017',
  'pages': '506--519',
  'booktitle': 'Proceedings of the 2017 ACM on Asia conference on computer and communications security',
  'author': 'Papernot, Nicolas and McDaniel, Patrick and Goodfellow, Ian and Jha, Somesh and Celik, Z Berkay and Swami, Ananthram',
  'title': 'Practical black-box attacks against machine learning',
  'ENTRYTYPE': 'inproceedings',
  'ID': '

In [110]:
# 2017_ryQbbFile


bib_dict = {}

# for v in iclr_arxiv_map["2017_ryQbbFile"].items():
k = "2018_B1ydPgTpW"
v = iclr_arxiv_map["2018_B1ydPgTpW"]

for i in range(0, 1):
    if v["found"] and v["arxivId"] in arxiv_source_files:
        try:
            f = "/home/singh_shruti/workspace/ICLR_arxiv_dump/" + v["arxivId"]
            tar = tarfile.open(f)
            file_mems = tar.getmembers()
            
            bib_files = []
            
            for t in file_mems:
                if t.name.find(".bib") > -1:
                    bib_files.append(t.name)
            
            if bib_files:
                print(bib_files)
                if len(bib_files) == 1:
                    
                    tar.extract(member=bib_files[0], path="/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison/temp_ext_dir/")
                    
                    new_loc = "/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison/temp_ext_dir/" + bib_files[0]
                    print(new_loc)
                    
                    with open(new_loc) as bibtex_file:
                        #bibtex_str = bibtex_file.read()
                        bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)
                    print("Read file")
#                     bib_database = bibtexparser.loads(bibtex_str)
#                     print("Loaded file")
                    
                    print(len(bib_database.entries))
                    bib_entries = []
                    for i, e in enumerate(bib_database.entries):
                        e["seq"] = i
                        bib_entries.append(e)
                    bib_dict[k] = bib_entries
#                 else:
                    #read multiple
        except Exception as ex:
            print(k)
            print(ex)
#             if ex != "file could not be opened successfully":
#                 print(repr(traceback.print_stack()))

['vinci_ML.bib']
/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison/temp_ext_dir/vinci_ML.bib


Entry type unknown not standard. Not considered.
Entry type online not standard. Not considered.


Read file
92


In [107]:
# bp = BibTexParser(interpolate_strings=False)
# bib_database = bp.parse(bibtex)

with open("/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison/temp_ext_dir/vinci_ML.bib") as bibtex_file:
#     bibtex_str = bibtex_file.read()
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)
print("Read file")
# bib_database = bibtexparser.loads(bibtex_str)

# files_i_want = ['path/to/file1','path/to/file2']

# tar = tarfile.open("bundle.tar")
# tar.extractall(members=[x for x in tar.getmembers() if x.name in files_i_want])

Entry type unknown not standard. Not considered.
Entry type online not standard. Not considered.


Read file


In [109]:
dir(bibtex_str)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_missing_from_crossref_entry',
 '_entries_dict',
 '_make_entries_dict',
 '_not_updated_by_crossref',
 'add_missing_from_crossref',
 'comments',
 'entries',
 'entries_dict',
 'entry_sort_key',
 'expand_string',
 'get_entry_dict',
 'get_entry_list',
 'load_common_strings',
 'preambles',
 'strings']

In [64]:
ls

1812.04606v3     DownloadArxivForICLR.ipynb
axcell_scripts/  extract_refs_from_arxiv.ipynb


In [65]:
pwd

'/home/singh_shruti/workspace/PaperAcceptancePrediction/shruti/meaningful_comparison'

In [66]:
mkdir temp_ext_dir

In [87]:
ls temp_ext_dir/

In [71]:
bib_files

[]

In [86]:
mkdir temp_ext_dir/

In [85]:
rm -r temp_ext_dir/

In [105]:
%%bash 
cat temp_ext_dir/vinci_ML.bib | grep -n2 oct

167-@INPROCEEDINGS{cho-al-emnlp14,
168-     author = {Cho, Kyunghyun and van Merri{\"{e}}nboer, Bart and G{\"{u}}l{\c c}ehre, Caglar and Bahdanau, Dzmitry and Bougares, Fethi and Schwenk, Holger and Bengio, Yoshua},
169:      month = oct,
170-      title = {Learning Phrase Representations using RNN Encoder--Decoder for Statistical Machine Translation},
171-  booktitle = {Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP)},
